In [ ]:
# %pip install earthengine-api geemap pycrs

In [ ]:
# %pip install earthengine-api --upgrade

In [ ]:
import ee
import geemap
from ipywidgets import Layout

Map = geemap.Map(layout = Layout(height='600px'))
Map.add_basemap('HYBRID')

In [ ]:
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
SFCC= {'bands': [ 'B8', 'B4', 'B3'], 'max': 3000}
roi = geemap.shp_to_ee("./Data/sangamner_shp/sangamner.shp")
s2 = ee.ImageCollection('COPERNICUS/S2').filterDate('2023-01-01', '2023-02-02').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)).filterBounds(roi)
Map.centerObject(roi, 10)

In [ ]:
s2

In [ ]:
s2_list = s2.toList(s2.size())
s2_list

In [ ]:
fcc_img = ee.Image(s2_list.get(2)).clip(roi)
Map.addLayer(fcc_img, SFCC, 'FCC')

In [ ]:
display(Map)

In [ ]:
image_index = [2,4,6,8]
image_collection = ee.ImageCollection([ee.Image(s2_list.get(i)).clip(roi) for i in image_index]).select(['B2', 'B3', 'B4', 'B8'])
image_collection

In [ ]:
def stackCollection(imgCollection: ee.ImageCollection) -> ee.Image:
    first = ee.Image(imgCollection.first())
    appendBands = lambda img, previous: ee.Image(previous).addBands(img)
    return ee.Image(imgCollection.iterate(appendBands, first))


In [ ]:
image_stack = stackCollection(image_collection)
image_stack

In [ ]:
agri = geemap.geojson_to_ee("./Data/featureCollections/Agricultural_Land.geojson")
barren = geemap.geojson_to_ee("./Data/featureCollections/Barren.geojson")
buildup = geemap.geojson_to_ee("./Data/featureCollections/BuildUp.geojson")
water = geemap.geojson_to_ee("./Data/featureCollections/Water.geojson")

In [ ]:
test = agri.first()

In [ ]:
feature = ee.Feature(test)
buffer_out = feature.buffer(128) #256x256m
buffer_out = buffer_out.bounds()
fthis= image_stack.clip(buffer_out)
Map.addLayer(buffer_out, {},"idk")
Map.centerObject(buffer_out)

In [ ]:
# Exactly 100x100 kernel
kernel = ee.Kernel.fixed(100, 100, [[1] * 100] * 100)

neighborhoodImage = image_stack.neighborhoodToArray(kernel)
samples = neighborhoodImage.sampleRegions(collection=agri)
exp_task = ee.batch.Export.table.toDrive(
    folder="GEE",
 collection=samples,
 fileFormat='TFRecord')

In [ ]:
# exp_task.start()

In [ ]:
Map